In [1]:
import requests
import pandas as pd

### Lista de urls

In [2]:
base_url = 'https://resultados.onpe.gob.pe/v1'

organizaciones_politicas_url = f'{base_url}/IN2020/agpcand/D40015'
tipo_elecciones_partido_url = f'{base_url}/IN2020/elecciones'

resumen_partido_url = f'{base_url}/IN2020/summary/actas'

resultados_elecciones_presidencialtes_url = f'{base_url}/IN2020/results/10/800000'
resultados_elecciones_congresales_url = f'{base_url}/IN2020/results/11/D43015'
resultados_elecciones_parlamento_andino_url = f'{base_url}/IN2020/results/12/800000'

ubigeos_regionales_por_partido_url = f'{base_url}/DE2020/ubigeos/01'
ubigeos_provinciales_por_partido_url = f'{base_url}/DE2020/ubigeos/03'
ubigeos_distritales_por_partido_url = f'{base_url}/DE2020/ubigeos/04'
resultados_elecciones_delegados_regional_url = f'{base_url}/DE2020/results/01'
resultados_elecciones_delegados_provincial_url = f'{base_url}/DE2020/results/03'
resultados_elecciones_delegados_distritales_url = f'{base_url}/DE2020/results/04'

iconos_organizaciones_url = 'https://resultados.onpe.gob.pe/IN2020/assets/iconos/iconos_reales'

### Organizaciones políticas - Detalles básicos

In [3]:
organizaciones_politicas = requests.get(organizaciones_politicas_url).json()['organizaciones']

In [4]:
organizaciones = []
for org in organizaciones_politicas:

    org_detalle = dict(
        organizacion=org['C_DESC_AGRUPOL'], 
        codigo=org['C_CODI_AGRUPOL'],
        codigo_corto=org['C_CODI_AGRUPOL'][-2:]
    )
    
    tipo_elecciones = requests.get(f'{tipo_elecciones_partido_url}/{org_detalle["codigo"]}').json()
    resumen_general = requests.get(f'{resumen_partido_url}/{org_detalle["codigo_corto"]}').json()
    
    org_detalle['tipo_elecciones'] = tipo_elecciones['elecciones']
    org_detalle['resumen_general'] = resumen_general

    organizaciones.append(org_detalle)

In [5]:
len(organizaciones)

24

In [6]:
organizaciones[0]

{'organizacion': 'ACCION POPULAR',
 'codigo': '00000002',
 'codigo_corto': '02',
 'tipo_elecciones': ['10', '12', '11'],
 'resumen_general': {'c_codi_proceso': '02',
  'descripcion': 'ACCION POPULAR',
  'n_elec_habil': '221,946',
  'n_num_mesas': '326',
  'locales': '127',
  'elecciones': [{'titulo': 'Afiliado Candidato',
    'sub_titulo': 'Modalidad: Fórmula Presidencial',
    'imagen': 'presidencial_ico'},
   {'titulo': 'Afiliado Candidato',
    'sub_titulo': 'Modalidad: Congreso de la República',
    'imagen': 'congreso_ico'},
   {'titulo': 'Afiliado Candidato',
    'sub_titulo': 'Modalidad: Parlamento Andino',
    'imagen': 'parlamento_ico'}]}}

### Resultados elecciones por organización política

In [7]:
for org in organizaciones:
    org['resultados_tipo_elecciones'] = []

    for tipo_eleccion in org['tipo_elecciones']:
        elecciones_delegados = ['01', '03', '04']

        if tipo_eleccion in elecciones_delegados:
            tipo = ''
            resultados_url = ''
            ubigeos_url = ''
            
            region_level = ''
            region_level_code = ''
            region_level_descripcion = ''
            region_level_ubigeos = []
            
            if tipo_eleccion == '01':
                tipo = 'delegados_regional'
                resultados_url = resultados_elecciones_delegados_regional_url
                ubigeos_url = ubigeos_regionales_por_partido_url
                region_level = 'departments'
                region_level_code = 'CDGO_DEP'
                region_level_descripcion = 'DESC_DEP'
            elif tipo_eleccion == '03':
                tipo = 'delegados_provincial'
                resultados_url = resultados_elecciones_delegados_provincial_url
                ubigeos_url = ubigeos_provinciales_por_partido_url
                region_level = 'provinces'
                region_level_code = 'CDGO_PROV'
                region_level_descripcion = 'DESC_PROV'
            elif tipo_eleccion == '04':
                tipo = 'delegados_distritales'
                resultados_url = resultados_elecciones_delegados_distritales_url
                ubigeos_url = ubigeos_distritales_por_partido_url
                region_level = 'districts'
                region_level_code = 'CDGO_DIST'
                region_level_descripcion = 'DESC_DIST'
            else:
                print('error al consultar tipo de eleccion no hay data')
                continue

            ubigeos = requests.get(f'{ubigeos_url}/{org["codigo"]}').json()
            region_level_ubigeos = ubigeos[region_level]

            resultados_detalle = dict(resultados=[], tipo=tipo, region_level=region_level, ubigeos=ubigeos)

            for level_ubigeo in region_level_ubigeos:
                resultados_por_nivel_region = requests.get(f'{resultados_url}/{level_ubigeo[region_level_code]}/{org["codigo"]}').json()
                nivel_region_detalle = dict(
                    nombre=level_ubigeo[region_level_descripcion],
                    codigo=level_ubigeo[region_level_code],
                    codigo_padre=level_ubigeo['CDGO_PADRE'],
                    resultados=resultados_por_nivel_region
                )
                resultados_detalle['resultados'].append(nivel_region_detalle)

            org['resultados_tipo_elecciones'].append(resultados_detalle)  

        else:
            tipo = ''
            url = ''
            if tipo_eleccion == '10':
                tipo = 'presidenciales'
                url = resultados_elecciones_presidencialtes_url
            elif tipo_eleccion == '11':
                tipo = 'congresales'
                url = resultados_elecciones_congresales_url
            elif tipo_eleccion == '12':
                tipo = 'parlamento_andino'
                url = resultados_elecciones_parlamento_andino_url

            resultados = requests.get(f'{url}/{org["codigo"]}').json()
            resultados_detalle = dict(resultados=resultados, tipo=tipo)
            org['resultados_tipo_elecciones'].append(resultados_detalle)

### Participación por organización política y tipo de elecciones

In [12]:
participacion_por_tipo = []
for org in organizaciones:
    for res in org['resultados_tipo_elecciones']:
        electores_habiles = 0
        electores_participaron = 0
        tipo = res['tipo']

        if type(res['resultados']) == dict:
            general_data = res['resultados']['generals']['generalData']
            electores_habiles += int(general_data['ELECTORES_HABIL'].replace(',', ''))
            electores_participaron += int(general_data['TOT_CIUDADANOS_VOTARON'].replace(',', ''))
        elif type(res['resultados']) == list:
            for sub_res in res['resultados']:
                general_data = sub_res['resultados']['generals']['generalData']
                electores_habiles += int(general_data['ELECTORES_HABIL'].replace(',', ''))
                electores_participaron += int(general_data['TOT_CIUDADANOS_VOTARON'].replace(',', ''))

        participacion_por_tipo.append(dict(
            organizacion = org['organizacion'],
            codigo = org['codigo'],
            tipo_eleccion = tipo,
            electores_habiles = electores_habiles,
            electores_participaron = electores_participaron,
        ))

In [13]:
participacion_por_tipo[0]

{'organizacion': 'ACCION POPULAR',
 'codigo': '00000002',
 'tipo_eleccion': 'presidenciales',
 'electores_habiles': 220351,
 'electores_participaron': 30142}

### Saving to csv

In [14]:
df = pd.DataFrame(participacion_por_tipo)

In [15]:
df.head()

,organizacion,codigo,tipo_eleccion,electores_habiles,electores_participaron
0,ACCION POPULAR,00000002,presidenciales,220351,30142
1,ACCION POPULAR,00000002,parlamento_andino,219856,29977
2,ACCION POPULAR,00000002,congresales,61743,9726
3,ALIANZA PARA EL PROGRESO,00000012,delegados_regional,224604,4807
4,AVANZA PAIS - PARTIDO DE INTEGRACION SOCIAL,00000016,delegados_regional,7881,362


In [17]:
df.to_csv('elecciones_internas_org_politicas_peru_2020.csv', index=False)